In [403]:
#Incorporating MN & SWUBS water use
import pandas as pd
import geopandas as gpd
import numpy as np

In [802]:
#Read in data
merged_wu_df = gpd.read_file('merged_WUdata_03212025.xlsx')
#Convert to geodataframe
merged_wu_gdf = gpd.GeoDataFrame(merged_wu_df, geometry = gpd.points_from_xy(x = merged_wu_df['utm_x'],y = merged_wu_df['utm_y']), crs = 'epsg:26915')

#Creating input 
        
merged_wu_gdf = merged_wu_gdf[merged_wu_gdf.columns[~merged_wu_gdf.columns.str.contains('200|199|198', regex = True)]] 
# merged_wu_gdf.drop(merged_wu_gdf.filter(regex='200|199|198', axis = 1), axis = 1, inplace = True) #Alternative using filter
    
merged_wu_gdf = merged_wu_gdf[~merged_wu_gdf['use_type'].str.contains('Dewatering')]
    
merged_wu_gdf =  merged_wu_gdf[merged_wu_gdf.loc[:,'use_2021_mg':'use_2010_mg'].sum(axis = 1) > 0]

output_df = merged_wu_gdf.loc[:,['well_number','utm_x','utm_y','well_depth_ft','geometry']]
output_df['use_2021-2010_mg'] = merged_wu_gdf.loc[:,'use_2021_mg':'use_2010_mg'].sum(axis =1 )
output_df = output_df.loc[:,['well_number','use_2021-2010_mg','well_depth_ft','utm_x','utm_y','geometry']].reset_index(drop = True)

output_df.to_csv('merged_wu_data.csv', index = False)


In [ ]:
#Read in Data
wuds_df = gpd.read_file('WUDS_download_03212025.xlsx')

#Filter out unnecessary columns
work_df = wuds_df.loc[:,['FROM Legacy Site Id', 'FROM Site Number', 'FROM Station Name', 'FROM DMS Latitude', 'FROM DMS Longitude', 'FROM Well Depth', 'FROM Altitude', 'Annual Value']]
#Adding in the value columns
work_df[wuds_df.loc[:, 'January Value':'December Value'].columns] = wuds_df.loc[:, 'January Value':'December Value']


group_df = work_df.groupby('FROM Legacy Site Id')
agg_func = dict.fromkeys(list(work_df.columns[7:20]),'sum')
agg_group_df = group_df.agg(agg_func)


work_df_nodupe = work_df[~work_df["FROM Legacy Site Id"].duplicated()].set_index('FROM Legacy Site Id')

output_df = work_df_nodupe.loc[:,'FROM Site Number':'FROM Altitude'].join(agg_group_df, how = 'outer',).reset_index() #Indexing to only join the first 7 unconflicted rows to the fixed output
output_df['month_sum_value'] = np.float64(output_df.loc[:,'January Value':'December Value']).sum(axis = 1)
test = output_df.columns.lower

output_df.to_csv('wuds_data.csv', index = False)

c:\Users\arichardet\AppData\Local\miniforge3\envs\cc-python-dev\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'WUDS_download_03212025.xlsx': 'Water Quantity' (default), 'Search Criteria'. Specify layer parameter to avoid this warning.
  result = read_func(


In [ ]:
test = list(map(str.lower.replace(' ','_').replace('from',''),output_df.columns)) # USE A LAMBDA FUNCTION TO MAKE THIS WORK
# def formatter(input):
#     input.str.lower().replace(' ','_').replace('from')
#     return input

# test = list(formatter,output_df.columns)

output_df.columns
test

AttributeError: 'method_descriptor' object has no attribute 'replace'